In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

LLM Sentence prediction - Short reproducible example using a handful of data to demonstrate process not performance.

Linear 1D layer model using embeddings

In [2]:
! git clone https://github.com/kpapdac/auto_spell.git

Cloning into 'auto_spell'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 105 (delta 32), reused 88 (delta 18), pack-reused 0
Receiving objects: 100% (105/105), 53.00 KiB | 4.42 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [4]:
# path = '/kaggle/input/llm-detect-ai-generated-text/'
# train_prompts = pd.read_csv(path + 'train_prompts.csv')#
# train_essays = pd.read_csv(path + 'train_essays.csv')
# test_essays = pd.read_csv(path + 'test_essays.csv')

In [6]:
train_essays.groupby(['prompt_id','generated']).nunique()

id  text
prompt_id generated           
0         0          707   707
          1            1     1
1         0          668   668
          1            2     2

In [2]:
import torch
import re
from auto_spell.auto_spell.src import model, optimize, textLoader, vocabulary
from torch.utils.data import DataLoader
from datasets import load_dataset

In [63]:
train_essays.shape

(1378, 4)

In [64]:
train_dataset, test_dataset = torch.utils.data.random_split(train_essays, [0.6, 0.4], 
                                                            torch.Generator().manual_seed(42))

In [3]:
def get_gen_nongen_sent(fullset):
    ser_sent = fullset.set_index('generated').text.str.split('.').explode()
    ser_sent_gen = [sent for sent in ser_sent[(ser_sent.index==1)].tolist() if sent!='']
    ser_sent_nongen = [sent for sent in ser_sent[(ser_sent.index==0)].tolist() if sent!=''][0:len(ser_sent_gen)]
    gen_text = [(1, \
             ser_sent_gen[i]) \
            for i in range(len(ser_sent_gen))]
    nongen_text = [(0, \
                ser_sent_nongen[i]) \
            for i in range(len(ser_sent_nongen))]
    return [gen_text, nongen_text]

In [4]:
def get_gen_nongen(fullset, underpresent_multiplier):
    gen_text = [(fullset[
          (fullset.generated==1)].reset_index().iloc[i,:]['generated'], \
             fullset[
          (fullset.generated==1)].reset_index().iloc[i,:]['text'][-1000:]) \
            for i in range(fullset[
                                        (fullset.generated==1)].shape[0])]*underpresent_multiplier
    nongen_text = [(fullset[
             (fullset.generated==0)].reset_index().iloc[i,:]['generated'], \
                fullset[#(train_essays.prompt_id==1) & \
             (fullset.generated==0)].reset_index().iloc[i,:]['text'][-1000:]) \
               for i in range(fullset[
                                       (fullset.generated==0)].shape[0])]
    return [gen_text, nongen_text]

Optimize in Small scale - step by step

In [6]:
import time
gen_nongen_train = [[(0,'This is my own text.'),
                     (0,'I wrote this myself.')],
                    [(1,'It is common knowledge that sky is blue.'),
                     (1,'Most available sources state that EV cars are still far from production.')]]
# gen_nongen_train = get_gen_nongen(train_dataset.dataset.loc[train_dataset.indices,:], 
#                                   int(0.1*train_dataset.dataset.loc[train_dataset.indices,:].shape[0]))
# gen_nongen_test = get_gen_nongen(test_dataset.dataset.loc[test_dataset.indices,:], 
#                                   int(0.1*test_dataset.dataset.loc[test_dataset.indices,:].shape[0]))
# gen_nongen_train = get_gen_nongen_sent(train_dataset.dataset.loc[train_dataset.indices,:])
text_label_train = gen_nongen_train[0] + gen_nongen_train[1]
voc = vocabulary.vocabulary()
voc.tokenizer = 'basic_english'
voc.get_vocab(text_label_train)
voc.set_text_pipeline()
voc.set_label_pipeline()
voc.get_voc_size()
voc.get_num_class(text_label_train)
num_embeddings = voc.vocab_size
embed_dim = 10
num_class = 2
mod = model.TextClassificationModel(num_embeddings, embed_dim, num_class)
prep = textLoader.prepareTextLabelLoaderLogisticNN(text_label_train, 'sms', 'label', voc.text_pipeline, voc.label_pipeline)
train_dataloader = DataLoader(text_label_train, batch_size=1, shuffle=False, collate_fn=prep.collate)
criterion = torch.nn.CrossEntropyLoss()
LR = 2
optimizer = torch.optim.SGD(mod.parameters(), lr=LR)

epochs = 5

# model(torch.tensor(voc.text_pipeline(text_label_train[0][1])), torch.tensor([0]))
# for name, param in model.named_parameters():
#     print(name, param.mean())
def train(epoch):
    for idx, (label, text, offsets) in enumerate(train_dataloader):
#         print(text)
        total_acc, total_count = 0, 0
        log_interval = 2
        start_time = time.time()
        optimizer.zero_grad()
        predicted_label = mod(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(mod.parameters(), 0.1)
        optimizer.step()
#         for name, param in model.named_parameters():
#             print(name, param.mean())
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
    #                 print(total_acc)
        running_loss =+ loss.item() * text.size(0)
#         print(running_loss)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
#             print('| epoch {:3d} | {:5d}/{:5d} batches '
#                 '| accuracy {:8.3f}'.format(epoch, idx, len(train_dataloader),
#                                             total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()
    return running_loss / len(train_dataloader)

def evaluate():
    mod.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(train_dataloader):
            predicted_label = mod(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

total_accu = None
loss_values = []
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    new_loss = train(epoch)
    loss_values.append(new_loss)
    print(f'Loss is: {new_loss}')
    accu_val = evaluate()
    if total_accu is not None and total_accu > accu_val:
        optimizer.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
        'valid accuracy {:8.3f} '.format(epoch,
                                        time.time() - epoch_start_time,
                                        accu_val))
    print('-' * 59)
# criterion = torch.nn.CrossEntropyLoss()
# LR = 0.03
# optimizer = torch.optim.SGD(model.parameters(), lr=LR)
# prep = textLoader.prepareTextLabelLoaderLogisticNN(text_label_train, 'sms', 'label', voc.text_pipeline, voc.label_pipeline)
# train_dataloader = DataLoader(text_label_train, batch_size=1, shuffle=False, collate_fn=prep.collate)


Loss is: 2.73775452375412
-----------------------------------------------------------
| end of epoch   1 | time:  0.01s | valid accuracy    0.500 
-----------------------------------------------------------
Loss is: 2.731295481324196
-----------------------------------------------------------
| end of epoch   2 | time:  0.00s | valid accuracy    0.750 
-----------------------------------------------------------
Loss is: 2.724315345287323
-----------------------------------------------------------
| end of epoch   3 | time:  0.00s | valid accuracy    1.000 
-----------------------------------------------------------
Loss is: 2.7162885665893555
-----------------------------------------------------------
| end of epoch   4 | time:  0.00s | valid accuracy    1.000 
-----------------------------------------------------------
Loss is: 2.7066370993852615
-----------------------------------------------------------
| end of epoch   5 | time:  0.00s | valid accuracy    1.000 
-------------------

After tuning

In [7]:
# gen_nongen_train = get_gen_nongen(train_dataset.dataset.loc[train_dataset.indices,:], 
#                                   int(0.1*train_dataset.dataset.loc[train_dataset.indices,:].shape[0]))
# gen_nongen_test = get_gen_nongen(test_dataset.dataset.loc[test_dataset.indices,:], 
#                                   int(0.1*test_dataset.dataset.loc[test_dataset.indices,:].shape[0]))
# gen_nongen_train = get_gen_nongen_sent(train_dataset.dataset.loc[train_dataset.indices,:])
gen_nongen_train = [[(0,'This is my own text.'),
                     (0,'I wrote this myself.')],
                    [(1,'It is common knowledge that sky is blue.'),
                     (1,'Most available sources state that EV cars are still far from production.')]]
# gen_nongen_test = get_gen_nongen_sent(test_dataset.dataset.loc[test_dataset.indices,:])
text_label_train = gen_nongen_train[0] + gen_nongen_train[1]
# text_label_test = gen_nongen_test[0] + gen_nongen_test[1]
voc = vocabulary.vocabulary()
voc.tokenizer = 'basic_english'
voc.get_vocab(text_label_train)
voc.set_text_pipeline()
voc.set_label_pipeline()
voc.get_voc_size()
voc.get_num_class(text_label_train)
num_embeddings = voc.vocab_size
embed_dim = 20
num_class = 2

mod = model.TextClassificationModel(num_embeddings, embed_dim, num_class)
criterion = torch.nn.CrossEntropyLoss()
LR = 2
optimizer = torch.optim.SGD(mod.parameters(), lr=LR)
prep = textLoader.prepareTextLabelLoaderLogisticNN(text_label_train, 'sms', 'label', voc.text_pipeline, voc.label_pipeline)
train_dataloader = DataLoader(text_label_train, batch_size=1, shuffle=False, collate_fn=prep.collate)
# test_dataloader = DataLoader(text_label_test, batch_size=1, shuffle=False, collate_fn=prep.collate)
opt = optimize.optimize(train_dataloader, train_dataloader, mod, criterion, optimizer, False)
# opt = optimize(train_dataloader, test_dataloader, model, criterion, optimizer, True)
opt.hypertuning(epochs=20)
res = [opt.predict(text_label_train[i][1], voc.text_pipeline) for i in range(len(text_label_train))]
print('Model test set: ', res.count(1), res.count(2)) #, res
real = [text_label_train[i][0] for i in range(len(text_label_train))]
print('Data test set: ', real.count(0), real.count(1))

Loss is: 2.753170371055603
-----------------------------------------------------------
| end of epoch   1 | time:  0.01s | valid accuracy    0.500 
-----------------------------------------------------------
Loss is: 2.745902180671692
-----------------------------------------------------------
| end of epoch   2 | time:  0.01s | valid accuracy    1.000 
-----------------------------------------------------------
Loss is: 2.7384449243545532
-----------------------------------------------------------
| end of epoch   3 | time:  0.01s | valid accuracy    1.000 
-----------------------------------------------------------
Loss is: 2.730266660451889
-----------------------------------------------------------
| end of epoch   4 | time:  0.01s | valid accuracy    1.000 
-----------------------------------------------------------
Loss is: 2.7207862436771393
-----------------------------------------------------------
| end of epoch   5 | time:  0.01s | valid accuracy    1.000 
------------------

In [8]:
opt.predict("Big cities have recently started doing ""CarFree"" days where no one is allowed to drive", voc.text_pipeline)

2

In [9]:
opt.predict("""The electrol college system is an unfair system, people don\'t have the right to select their own president, they dont have the right to select a president.""", voc.text_pipeline)

2